🌱 인프런 📚 모두의 한국어 텍스트 분석과 자연어처리 with 파이썬 🐍 https://inf.run/FX4TP


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/python-text-analysis/blob/main/0502-kogpt2-input.ipynb)

## Hugging Face의 transformers 라이브러리를 사용하여 한국어 GPT-2 모델을 활용하여 문장 생성하기

In [ ]:
# transformers는 Hugging Face에서 제공하는 자연어처리 라이브러리로, 다양한 사전학습 언어모델을 제공하고 있습니다.
# accelerate는 Hugging Face에서 제공하는 학습 가속 라이브러리로, PyTorch와 함께 사용할 수 있습니다.
# 이를 이용하면 단일 노드의 다중 GPU 학습을 지원하고, 학습시간을 단축할 수 있습니다.
!pip install transformers --upgrade --q
!pip install accelerate --q

## koGPT2
* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)

* tokenizers 패키지의 Character BPE tokenizer로 학습되었습니다.


* 사전 크기는 51,200 이며 대화에 자주 쓰이는 아래와 같은 이모티콘, 이모지 등을 추가하여 해당 토큰의 인식 능력을 올렸습니다.
* 또한 <unused0> ~ <unused99>등의 미사용 토큰을 정의해 필요한 테스크에 따라 자유롭게 정의해 사용할 수 있게 했습니다.

* BPE(Byte Pair Encoding)은 하나의 텍스트를 토큰으로 분리하기 위한 subword 기반 알고리즘이며, 유사한 알고리즘으로는 WordPiece, SentencePiece 등이 있습니다. 이들 알고리즘은 모두 텍스트를 일정 크기의 subword 단위로 쪼개서 처리하는 과정에서 OOV(Out-Of-Vocabulary) 문제를 해결하고자 개발된 알고리즘입니다.
    * Wordpiece 모델에서 Out-of-vocabulary(OOV) 문제를 해결하기 위해, 모든 단어를 서브워드(subword) 단위로 분할하여 처리합니다. 즉, 모든 단어를 이루는 subword들의 조합으로 단어를 대체합니다. 예를 들어 "playing"이라는 단어를 Wordpiece 모델에 넣으면 "play"와 "##ing"으로 분할하여 처리합니다. "##"은 서브워드를 나타내는 특별한 기호입니다. 이렇게 단어를 subword들의 집합으로 분할하여 처리함으로써, 이전에 OOV 문제로 처리할 수 없었던 단어를 분할하여 처리할 수 있게 되어 모델의 성능을 향상시킬 수 있습니다.
    * SentencePiece는 Google에서 개발한 오픈소스 기반의 자연어 처리 도구입니다. BPE(Byte Pair Encoding)와 Unigram Language Model Tokenizer(ULM)를 적용할 수 있으며, 모델의 입력으로 사용할 수 있는 단어 집합을 만들기 위해 사용됩니다.
    * SentencePiece는 형태소 분석 및 품사 태깅을 수행하지 않고, 문자열을 작은 서브워드(subword)로 분할합니다. 이를 통해, OOV(Out of Vocabulary) 문제를 완화하고, 모델의 성능을 높일 수 있습니다. 또한, SentencePiece는 대규모 텍스트 데이터에서 자동으로 언어 모델을 학습하므로, 사전에 훈련된 모델을 사용하여 적은 데이터로도 효과적인 자연어 처리를 수행할 수 있습니다.
    * SentencePiece는 PyTorch, TensorFlow, C++, Java, Python 등에서 사용할 수 있습니다. 또한, 다양한 언어(한국어, 영어, 일본어 등)를 지원하며, 사용자가 직접 모델을 학습할 수 있도록 API를 제공합니다.



## PreTrainedTokenizerFast

* tokenizer를 생성합니다. from_pretrained 함수를 사용하여 사전 학습된 한국어 GPT-2 모델을 불러옵니다. 이때, 시작 토큰, 종료 토큰, 알 수 없는 토큰, 패딩 토큰, 마스크 토큰 등을 설정할 수 있습니다.

* 마스크 토큰은 자연어 처리에서 문장 내 일부 단어를 마스킹하여 모델이 이를 예측하는 작업입니다. 마스킹 작업은 모델이 문장 내에서 해당 단어가 빠진 문맥을 이해하고, 문맥에 맞는 적절한 단어를 예측하는 능력을 강화시키는 데에 유용합니다.

* 마스크 토큰은 대부분의 transformer 모델에서 사용되며, BERT, RoBERTa, GPT 등에서 사용됩니다. 대표적으로 BERT에서는 15%의 단어를 마스킹하며, 이 중 80%는 [MASK] 토큰으로, 10%는 랜덤한 단어로, 10%는 원래 단어로 대체됩니다.

* 한국어 GPT-2 모델에서도 마스크 토큰을 지원하며, tokenizer에서 mask_token 옵션을 사용하여 마스크 토큰을 설정할 수 있습니다. 마스크 토큰을 사용하는 것은 모델의 학습 능력을 향상시키는 데에 큰 도움을 줄 수 있습니다.


* bos_token : 토크나이저에서 사용되는 특수 토큰 중 하나로, 문장의 시작을 나타내는 토큰입니다. BOS는 "Beginning of Sentence"의 약자로, 이 토큰이 삽입된 위치가 문장의 시작임을 나타냅니다. BOS 토큰은 토크나이징된 시퀀스의 맨 앞에 추가됩니다. 예를 들어, "Hello, world!"라는 문장을 BERT 토크나이저로 토큰화하면 "[CLS] Hello, world! [SEP]"와 같이 BOS 토큰이 "[CLS]"로 추가되어 있습니다.
* eos_token: 문자열의 끝을 나타내는 토큰입니다.
* unk_token: 알 수 없는 토큰입니다. 모델이 학습하지 않은 단어를 나타낼 때 사용됩니다.
* pad_token: 시퀀스 패딩에 사용되는 토큰입니다. 시퀀스의 길이를 일정하게 맞추기 위해 사용됩니다.
* mask_token: 마스킹 토큰입니다. 모델의 입력 시퀀스 중 일부 토큰을 마스킹하여 학습에 사용됩니다.

* [Tokenizer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast)
* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)

In [ ]:
# PreTrainedTokenizerFast


In [ ]:
raw_text = "안녕하세요. 한국어 GPT-2 입니다.😤"

In [ ]:
# tokenize(): 문장을 token으로 나눠서 반환
# tokens

In [ ]:
# convert_tokens_to_ids(): token 목록을 받아서 token_id 목록을 반환
# token_ids

In [ ]:
# convert_ids_to_tokens(): token_id 목록을 받아서 token 목록을 반환
# converted_tokens

In [ ]:
# convert_tokens_to_string(): token 목록을 받아서 문장으로 반환
# converted_text

In [ ]:
# encode(): token으로 자르기 + token을 token_id로 변환하여 반환
# = tokenize() + convert_tokens_to_ids()


In [ ]:
# tokenizer가 갖고 있는 특별한 token들을 확인
tokenizer.bos_token, tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token, tokenizer.mask_token

In [ ]:
# tokenizer가 갖고 있는 특별한 token들의 token_id를 확인
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.mask_token_id

In [ ]:
# UNK: 학습하지 못한 입력을 대체
tokenizer.tokenize("뷁")

## 다른 문장 토큰화 적용

In [ ]:
sample_text = """
    마스킹은 모델 학습에서 매우 유용한 기술 중 하나입니다.
    이를 통해 모델은 입력 시퀀스의 일부를 무작위로 마스킹하여 누락된 토큰을 예측하도록 학습할 수 있습니다.
    이렇게 하면 모델은 입력 시퀀스에서 누락된 정보를 추론하는 능력을 향상시키며,
    더 일반화된 학습이 가능해집니다.
    예를 들어, 기계 번역에서는 마스킹을 사용하여 모델이 문장의 일부 단어를 예측하도록 학습시킬 수 있습니다.
    이를 통해 모델은 문장의 구조와 문맥을 이해하고, 번역의 정확도를 향상시킬 수 있습니다.
    또한, 마스킹은 모델이 오버피팅(overfitting)을 피하는 데도 도움이 됩니다.
    오버피팅은 모델이 학습 데이터에 지나치게 적합하여 새로운 데이터에 대한 일반화 능력이 부족한 상태를 의미합니다.
    마스킹을 통해 모델은 입력 시퀀스의 랜덤한 부분을 누락시킴으로써 일반화 능력을 향상시키고, 오버피팅을 방지할 수 있습니다.
    따라서, 마스킹은 모델 학습에서 중요한 역할을 합니다. 이를 통해 모델은 누락된 정보를 추론하는 능력을 강화하고,
    오버피팅을 방지하며, 더 일반화된 학습이 가능해집니다.
    """

In [ ]:
# tokenizer.tokenize


In [ ]:
# tokenizer.encode


## GPT2LMHeadModel

 GPT2LMHeadModel 클래스를 사용하여 모델을 불러옵니다. from_pretrained 함수를 사용하여 사전 학습된 한국어 GPT-2 모델을 불러옵니다.

GPT2LMHeadModel는 OpenAI에서 개발한 언어 모델인 GPT-2(GPT-2 Language Model)을 파이토치(PyTorch)로 구현한 모델입니다.

GPT-2는 Transformer 아키텍처를 기반으로 한 언어 모델로, 입력된 문장에 대해 다음 단어를 예측하는 과정에서 이전 단어들의 상호작용을 반영합니다. Transformer는 주로 자연어처리 분야에서 사용되는 딥러닝 모델 중 하나로, 이전의 RNN(Recurrent Neural Network) 모델에 비해 계산 속도가 빠르며, 병렬 처리가 용이하여 학습 시간이 단축됩니다.

GPT2LMHeadModel은 Transformer 모델의 블록인 GPT2Block을 12개 연결한 transformer와, 언어 모델의 출력층인 lm_head로 구성됩니다. transformer는 입력된 텍스트 시퀀스를 이용해 텍스트를 임베딩하고, 자기어텐션(self-attention)과 전방향(feedforward) 신경망으로 구성된 블록인 GPT2Block을 통해 시퀀스의 정보를 추출합니다. lm_head는 transformer의 출력을 입력받아 다음 단어를 예측하는 과정에서 사용됩니다.

따라서 GPT2LMHeadModel는 입력된 텍스트를 이용해 다음 단어를 예측하는 언어 모델입니다. 이를 통해 대화 시스템, 번역, 요약 등의 다양한 자연어 처리 태스크를 수행할 수 있습니다.

* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)
* [OpenAI GPT2](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/gpt2#transformers.GPT2LMHeadModel)

In [ ]:
# GPT2LMHeadModel.from_pretrained
# skt/kogpt2-base-v2



### GPT2Model 사용하기

* GPT2LMHeadModel은 GPT-2 모델의 언어 모델링 버전으로, 입력 시퀀스에 대해 다음 단어를 예측하는 작업을 수행합니다.
* 이 모델은 GPT2Model과 LM Head로 구성됩니다.
* GPT2Model은 GPT-2 아키텍처를 구현한 모듈이며, 입력 시퀀스를 받아 transformer 레이어를 거치며 출력값을 내보냅니다. 이 모델은 다음과 같은 하위 모듈들로 구성됩니다.
* WTE(Word Token Embedding)는 각 단어를 고정 크기의 벡터로 변환하는 기술로, 단어를 벡터 공간에 임베딩하여 자연어 처리 모델에 입력으로 제공합니다. 이를 통해 단어 간 유사도를 측정하거나, 텍스트 데이터의 차원을 줄이는 등 다양한 자연어 처리 작업에서 사용됩니다. WTE는 일반적으로 원-핫 인코딩 방법과 달리, 단어의 의미를 나타내는 잠재적인 구조를 학습하도록 설계됩니다. 이를 위해 WTE는 주로 언어 모델에 사용되는 신경망 아키텍처인 임베딩 레이어와 결합하여 사용됩니다.

    * wte: 입력 토큰의 임베딩 테이블입니다.
    * wpe: 입력 토큰의 위치 임베딩 테이블입니다.
    * drop: 입력 토큰의 임베딩과 위치 임베딩에 대한 드롭아웃을 수행합니다.
    * h: GPT-2 블록들의 스택으로 구성된 transformer 레이어입니다.
    * ln_f: 출력값에 대한 Layer Normalization을 수행합니다.


트랜스포머 모델에서 Layer Normalization은 다음과 같은 수식으로 정의됩니다.

$$\text{LayerNorm}(x_i) = \gamma \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$


트랜스포머 모델에서 Layer Normalization은 각 레이어의 입력을 정규화하는 기법 중 하나입니다. 각 레이어의 입력은 해당 레이어에서 연산이 수행되기 전에 정규화됩니다. 이러한 정규화는 미니배치 내에서 각 샘플의 평균과 분산을 사용하여 수행됩니다. 즉, 레이어별로 입력에 대한 평균과 분산을 구하고, 이를 사용하여 입력을 정규화합니다.

Layer Normalization은 Batch Normalization과 비슷한 목적을 가지지만, Batch Normalization은 미니배치 단위로 입력을 정규화하는 반면, Layer Normalization은 레이어 내에서 입력을 정규화합니다. 이러한 차이점으로 인해 Layer Normalization은 Batch Normalization과 비교해 작은 미니배치나 긴 시퀀스의 경우에도 안정적으로 학습할 수 있는 장점이 있습니다.

Layer Normalization은 RNN과 같이 시퀀스 모델링에서 성능 향상에 효과적인 것으로 알려져 있습니다. 이는 시퀀스 모델링에서 배치 정규화를 적용하기 어려운 점과 관련이 있습니다. 배치 정규화는 미니배치 크기에 따라 성능이 크게 달라질 수 있는 반면, Layer Normalization은 샘플 단위로 계산하기 때문에 미니배치 크기에 영향을 받지 않습니다. 또한 Layer Normalization은 모델의 불안정성을 줄이고 학습 속도를 빠르게 하는 효과도 있습니다.


여기서 $x$는 입력 텐서, $\mu$와 $\sigma^2$는 평균과 분산으로, 미니배치 내에서 샘플별로 계산됩니다. $\gamma$와 $\beta$는 학습 가능한 스케일(scale)과 시프트(shift) 매개변수입니다. $\epsilon$은 작은 값으로 분모가 0이 되는 것을 방지합니다.



* $x_i$ : $i$번째 입력 특성 값
* $\mu$ : 평균
* $\sigma$ : 표준편차
* $\epsilon$ : 작은 값으로 분모가 0이 되는 것을 방지하기 위한 값
* $\gamma$ : 스케일 조정 파라미터
* $\beta$ : 이동 조정 파라미터
* $\text{LN}(x_i)$ : 정규화된 $i$번째 입력 특성 값




### GPT-2 블록

은 self-attention과 MLP 레이어로 구성되어 있습니다. 이 중 self-attention은 입력 시퀀스 내에서 다양한 위치들 간의 상호작용을 수행하여, 특정 위치의 출력값이 전체 시퀀스의 컨텍스트에 따라 동적으로 결정되는 효과를 가집니다.

#### LM Head

LM Head는 transformer 레이어의 출력값을 받아, 다음 단어 예측을 수행하는 선형 레이어입니다. 이 레이어는 단어의 분포를 출력값으로 내보내며, 이 분포에서 가장 높은 확률을 가지는 단어가 모델의 예측값이 됩니다.


* https://jalammar.github.io/illustrated-transformer/

<img src="https://nlpinkorean.github.io/images/transformer/transformer_resideual_layer_norm_3.png">

In [ ]:
# model

* 참고 :
* Conv1D와 Dot Product 모두 벡터 간의 곱셈을 수행합니다. 그러나 Conv1D는 한 쪽 벡터를 이동시키면서 다른 쪽 벡터와의 내적을 계산하는 반면, Dot Product는 두 벡터의 요소 간 곱의 합으로 구성됩니다.

* Conv1D는 커널(kernel)을 이동시키면서 입력 벡터와의 내적을 계산합니다. 이 때 커널의 크기는 고정되어 있지만, 이동하는 위치에 따라 결과가 달라질 수 있습니다. 이를 통해 입력 벡터 내의 패턴을 인식하거나 특징(feature)을 추출하는 등의 역할을 수행할 수 있습니다.

* 반면, Dot Product는 두 벡터의 요소 간 곱의 합으로 구성됩니다. 이를 통해 두 벡터 간의 유사도를 측정하거나, 각 요소가 독립적인 경우에 대해 연산을 수행할 수 있습니다.

* 따라서 Conv1D와 Dot Product는 비슷한 연산이지만, Conv1D는 입력 벡터 내에서 패턴을 인식하는 데에 적합하고, Dot Product는 벡터 간의 유사도 측정 등에 적합합니다.

## 생성
generate 함수를 사용하여 모델을 이용하여 문장을 생성합니다. input_ids에 입력으로 넣을 텍스트를 tokenize한 결과를 Tensor 형태로 입력합니다. max_length는 최대 생성 문장 길이, repetition_penalty는 반복되는 단어를 억제하는 페널티 값입니다. pad_token_id, eos_token_id, bos_token_id는 tokenizer에서 설정한 값과 동일하게 입력해주어야 합니다.

마지막으로, decode 함수를 사용하여 생성된 Tensor 값을 텍스트로 디코딩합니다.

* tokenizer.encode(text, return_tensors='pt'): 주어진 text를 tokenizer를 활용하여 토큰화하고, 이를 pytorch tensor 형태로 변환합니다.
* model.generate(): pytorch tensor 형태로 변환된 input_ids를 GPT2 모델에 입력으로 넣어 자연어 생성을 수행합니다. max_length는 생성할 문장의 최대 길이를 의미하며, repetition_penalty는 반복을 방지하는데 사용되는 계수입니다. pad_token_id, eos_token_id, bos_token_id는 각각 패딩, 문장 끝, 문장 시작을 의미하는 토큰의 id를 나타냅니다. use_cache는 이전 단계의 계산 결과를 재사용할지 여부를 나타냅니다.
* tokenizer.decode(gen_ids[0]): GPT2 모델의 출력값인 gen_ids를 tokenizer를 활용하여 토큰을 텍스트로 디코딩합니다.


In [ ]:
text = '꾸준한 운동은 '


In [ ]:
# text를 받아 생성한 문장을 반환하는 함수
# def generate_text(text):
#     return generated

In [ ]:
text = '봄 여행지 추천?'
generate_text(text)

In [ ]:
text = '오늘의 점심 식사는?'
generate_text(text)

In [ ]:
text = '샐러드 레시피'
generate_text(text)

In [ ]:
generate_text("파이썬 코딩 공부 방법 추천")